In [1]:
import os
os.chdir('../src')
import ctypes
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as colors
import pandas as pd


In [2]:
from ulxlc import ULXLC
from constants import params_default


In [3]:
#lc_boost = lc_zero_incl_max_flux / lc_max
#       2 = 100 / 50

In [4]:
incls  = np.arange(0,91,1)
dincls = np.arange(0,46,1)
thetas = np.arange(0,46,1)

N_incls  = len(incls)
N_dincls = len(dincls)
N_thetas = len(thetas)

N_tot = N_incls*N_dincls*N_thetas

save_par = ['dincl', 'theta', 'incl', 'lc_min', 'lc_max', 'lc_boost']
N_save_par = len(save_par)

c_arr = (ctypes.c_double*N_save_par*N_tot)()

In [ ]:
ulxlc = ULXLC()
ulxlc.libc.lc_boost(ulxlc.lc_t, ulxlc.lc_nt, ulxlc.lc_flux, c_arr, N_tot, N_save_par)
arr = np.ctypeslib.as_array(c_arr)

In [ ]:
df = pd.DataFrame(arr)
df.columns = [*save_par]
df

In [ ]:
df

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
Z = df['lc_boost']
ax.scatter(df['incl'], df['dincl'], df['theta'], c=Z, alpha=0.5, norm=colors.LogNorm(), s=0.5)
ax.set_xlabel('incl')
ax.set_ylabel('dincl')
ax.set_zlabel('theta')
plt.show()

In [ ]:
df2

In [ ]:
theta = 30
df2 = df[df['theta'] == theta]
df2 = df2[df2['lc_boost'] < 1000]
df2 = df2[df2['lc_boost'] > 0]

plt.figure(figsize=(6,4))
plt.title(fr'theta={theta}$^{{\circ}}$ | L Reduction')
plt.scatter(df2['incl'], df2['dincl'], c=df2['lc_boost'])
plt.xlabel('Inclination (i)')
plt.ylabel('Precessional angle ($\Delta i$)')
plt.colorbar()

In [ ]:
thetas = [10,25,45]
n_thetas = len(thetas)
#w = 6.93
w = 16
h = w/n_thetas
fig, ax = plt.subplots(nrows=1, ncols=n_thetas, sharey=True, sharex=True)
fig.set_size_inches([w,h])

cmap = plt.cm.get_cmap('hot')

for i, theta in enumerate(thetas):
    df2 = df[df['theta'] == theta]
    df2 = df2[df2['lc_boost'] < 100]
    df2 = df2[df2['lc_boost'] > 2]
    
    
    ax[i].set_title(fr'$\theta / 2$ = {theta}$^{{\circ}}$')
    #ax[i].scatter(df2['incl'], df2['dincl'], c=df2['lc_boost'], norm=colors.LogNorm())
    tri = ax[i].tricontourf(df2['incl'], df2['dincl'], df2['lc_boost'], levels=100, cmap=cmap)
    
ax[0].set_xlim(0,90)
ax[0].set_ylim(0,45)
ax[0].set_xticks(np.arange(0,90,15))

ax[1].set_xlabel('Inclination (i)')
ax[0].set_ylabel('Precessional angle ($\Delta i$)')
plt.subplots_adjust(hspace=0, wspace=0)
fig.colorbar(tri, ax=ax[-1], orientation='vertical')

In [ ]:
df2